In [1]:
import os
import pandas
import numpy as np

# Get and manipulate Brainspan data (https://github.com/PennLINC/Vogel_PLS_Tx-Space/blob/main/NB06_BrainSpanValidation.ipynb)

In [5]:
os.system('wget https://www.brainspan.org/api/v2/well_known_file_download/267666525 --no-check-certificate')

--2023-09-07 08:47:03--  https://www.brainspan.org/api/v2/well_known_file_download/267666525
Resolving www.brainspan.org (www.brainspan.org)... 163.123.189.39
Connecting to www.brainspan.org (www.brainspan.org)|163.123.189.39|:443... connected.
  Unable to locally verify the issuer's authority.
HTTP request sent, awaiting response... 200 OK
Length: 62218535 (59M) [application/zip]
Saving to: ‘267666525’

     0K .......... .......... .......... .......... ..........  0%  281K 3m36s
    50K .......... .......... .......... .......... ..........  0%  407K 3m3s
   100K .......... .......... .......... .......... ..........  0%  810K 2m27s
   150K .......... .......... .......... .......... ..........  0%  817K 2m8s
   200K .......... .......... .......... .......... ..........  0% 61.4M 1m43s
   250K .......... .......... .......... .......... ..........  0%  816K 98s
   300K .......... .......... .......... .......... ..........  0%  813K 95s
   350K .......... .......... .......... ....

256

In [2]:
bspan_dir = '/dagher/dagher11/filip/Obesity_maps/data/brainspan/'

In [3]:
df = pandas.read_csv(os.path.join(bspan_dir,'expression_matrix_edited.csv'),header=None) # expression data
rmd = pandas.read_csv(os.path.join(bspan_dir,'rows_metadata.csv')) # rows metadata
cmd = pandas.read_csv(os.path.join(bspan_dir,'columns_metadata.csv')) # columns metadata
print (len(rmd.gene_symbol.unique()))
print (df.shape,rmd.shape)

# address duplicates
rmd = rmd.drop_duplicates(subset=['gene_symbol'], keep='last')

labels = rmd.index.values
df = df.loc[df.index.intersection(labels)]
print (df.shape,rmd.shape)

47808
(52376, 524) (52376, 5)
(47808, 524) (47808, 5)


In [4]:
gdf = pandas.DataFrame(cmd,copy=True)

In [5]:
order = ['8 pcw', '9 pcw', '12 pcw', '13 pcw', '16 pcw', 
         '17 pcw', '19 pcw', '21 pcw','24 pcw', '25 pcw', 
         '26 pcw', '35 pcw', '37 pcw','4 mos','10 mos',
         '1 yrs', '2 yrs', '3 yrs','4 yrs','8 yrs',
        '11 yrs', '13 yrs', '15 yrs', '18 yrs', '19 yrs',
        '21 yrs', '23 yrs','30 yrs', '36 yrs', '37 yrs',
        '40 yrs']

# match donors to samples
ABCD = ['4 yrs','8 yrs','11 yrs','13 yrs','15 yrs']
HCP = ['18 yrs', '19 yrs','21 yrs', '23 yrs','30 yrs', '36 yrs']

divs = [ABCD, HCP]
names = ['ABCD','HCP']
bdivs = dict(zip(names,divs))
for name, div in bdivs.items():
    gdf.loc[gdf[gdf.age.isin(div)].index,'Age_Stage'] = name

/tmp/ipykernel_4447/1586796574.py:17: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'ABCD' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  gdf.loc[gdf[gdf.age.isin(div)].index,'Age_Stage'] = name


In [6]:
# brain divisions

DKT =  ['A1C','DFC','IPC','ITC','M1C','MFC','OFC','S1C','STC','V1C','VFC']
             
divs = [DKT]
names = ['DKT']
bdivs = dict(zip(names,divs))
for name, div in bdivs.items():
    gdf.loc[gdf[gdf.structure_acronym.isin(div)].index,'MacroStructure'] = name

/tmp/ipykernel_4447/1709443361.py:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'DKT' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  gdf.loc[gdf[gdf.structure_acronym.isin(div)].index,'MacroStructure'] = name


In [7]:
# Filter df_ABCD and df_HCP
mask_ABCD = (gdf['Age_Stage'] == 'ABCD') & (gdf['MacroStructure'] == 'DKT')
df_ABCD = df.loc[:, mask_ABCD.values]

mask_HCP = (gdf['Age_Stage'] == 'HCP') & (gdf['MacroStructure'] == 'DKT')
df_HCP = df.loc[:, mask_HCP.values]

# Combine 'Macrostructure' and 'Structure_Acronym' into a single key for grouping
grouping_key_ABCD = gdf.loc[mask_ABCD, 'MacroStructure'] + '_' + gdf.loc[mask_ABCD, 'structure_acronym']
grouping_key_HCP = gdf.loc[mask_HCP, 'MacroStructure'] + '_' + gdf.loc[mask_HCP, 'structure_acronym']

# Group by the combined key and calculate the mean for each group
df_ABCD_grouped = df_ABCD.groupby(grouping_key_ABCD, axis=1).mean()
df_HCP_grouped = df_HCP.groupby(grouping_key_HCP, axis=1).mean()

# Rename the columns
df_ABCD_grouped.columns = ['{}_ABCD'.format(col) for col in df_ABCD_grouped.columns]
df_HCP_grouped.columns = ['{}_HCP'.format(col) for col in df_HCP_grouped.columns]

# df_ABCD_grouped and df_HCP_grouped now contain the averaged data for each combination

/tmp/ipykernel_4447/427614613.py:13: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  df_ABCD_grouped = df_ABCD.groupby(grouping_key_ABCD, axis=1).mean()
/tmp/ipykernel_4447/427614613.py:14: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  df_HCP_grouped = df_HCP.groupby(grouping_key_HCP, axis=1).mean()


In [8]:
df_ABCD_grouped['gene']=rmd['gene_symbol']
df_HCP_grouped['gene']=rmd['gene_symbol']

# Get list of AHBA genes that overlaps with brainspan genes

In [134]:
AHBA_genes = pandas.read_csv('/dagher/dagher11/filip/Obesity_maps/data_old/abagenes.csv', header=None) # expression data


In [171]:
set1 = set(AHBA_genes[0])
set2 = set(rmd['gene_symbol'])
overlap_strings = list(set1.intersection(set2))
np.savetxt("/dagher/dagher11/filip/Obesity_maps/data/aba_brainspan_genes.csv",
        overlap_strings,
        delimiter =", ",
        fmt ='% s')

rmd

In [11]:
np.savetxt("/dagher/dagher11/filip/Obesity_maps/data/brainspan_genes.csv",
        rmd['gene_symbol'],
        delimiter =", ",
        fmt ='% s')

In [154]:
df_ABCD_grouped_filtered=df_ABCD_grouped[df_ABCD_grouped['gene'].isin(overlap_strings)]
df_HCP_grouped_filtered=df_HCP_grouped[df_HCP_grouped['gene'].isin(overlap_strings)]

In [157]:
data_dict_ABCD = {'df_ABCD_grouped_filtered': df.to_dict()}
data_dict_HCP = {'df_HCP_grouped_filtered': df.to_dict()}


# Save as matlab files

In [10]:
df_ABCD_grouped.to_csv('/dagher/dagher11/filip/Obesity_maps/data/ABCD_braispan_all.csv')
df_HCP_grouped.to_csv('/dagher/dagher11/filip/Obesity_maps/data/HCP_braispan_all.csv')

In [170]:
df_ABCD_grouped_filtered.to_csv('/dagher/dagher11/filip/Obesity_maps/data/ABCD_braispan.csv')
df_HCP_grouped_filtered.to_csv('/dagher/dagher11/filip/Obesity_maps/data/HCP_braispan.csv')

In [165]:
df_ABCD_grouped_filtered

,DKT_A1C_ABCD,DKT_DFC_ABCD,DKT_IPC_ABCD,DKT_ITC_ABCD,DKT_M1C_ABCD,DKT_MFC_ABCD,DKT_OFC_ABCD,DKT_S1C_ABCD,DKT_STC_ABCD,DKT_V1C_ABCD,DKT_VFC_ABCD,gene
0,3.698800,3.997434,4.263902,5.789062,4.206122,5.034047,3.743397,5.015604,5.523693,3.933958,4.977298,TSPAN6
2,12.894400,11.038067,14.412805,16.470940,13.256018,11.608446,11.370208,16.509985,16.609682,12.985141,12.894081,DPM1
3,1.936599,1.619315,1.927586,2.126737,2.151824,1.684136,2.008659,2.125423,1.959122,1.806423,1.870815,SCYL3
4,0.637432,0.541984,0.597468,0.706927,0.724992,0.509843,0.671367,0.690023,0.653861,0.566198,0.592809,C1orf112
5,1.029065,1.128451,1.182775,0.859087,1.417516,1.071546,1.052766,1.294967,1.225799,1.350998,1.200607,FGR
...,...,...,...,...,...,...,...,...,...,...,...,...
52349,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,SHOX
52352,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,CSF2RA
52353,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,CRLF2
52354,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,ZBED1
